In [0]:
import numpy as np 
import pandas as pd 
import cv2, pickle
from PIL import Image 
import os
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.applications import InceptionResNetV2
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [0]:
train = pickle.load(open("/content/drive/My Drive/Colab Notebooks/dl_comptetion/dataset/train.npy",'rb'))
test = pickle.load(open("/content/drive/My Drive/Colab Notebooks/dl_comptetion/dataset/test.npy",'rb'))
train = np.array(train)
test = np.array(test)
train_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/dl_comptetion/dataset/train.csv")

encoder = LabelEncoder()
encoder.fit(train_data["target"])
encoded_Y = encoder.transform(train_data["target"])
y_labels = np_utils.to_categorical(encoded_Y)


In [0]:
IMG_WIDTH=256
IMG_HEIGHT=256
BATCH_SIZE = 5

In [0]:
from tensorflow.keras.models import Model
import tensorflow.keras as keras

resnet = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3),pooling='avg')

output = resnet.layers[-1].output
output = tf.keras.layers.Flatten()(output)
resnet = Model(resnet.input, output)

219062272/219055592 [==============================] - 2s 0us/step


In [0]:
res_name = []
for layer in resnet.layers:
    res_name.append(layer.name)

set_trainable = False
for layer in resnet.layers:
    if layer.name in res_name[-447:]:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout

model = Sequential()
model.add(resnet)
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(8, activation='softmax'))


adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8,
                                              restore_best_weights=False
                                              )

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.2,
                                   patience=4,
                                   verbose=1,
                                   min_delta=5*1e-3,min_lr = 5*1e-7,
                                   )

In [0]:
model.compile(optimizer = adam, 
              loss = 'categorical_crossentropy', 
              metrics=['accuracy',tfa.metrics.F1Score(num_classes=8)])

In [0]:
model.fit(train, y_labels,steps_per_epoch=np.ceil(float(train.shape[0]) / float(BATCH_SIZE)),
                        epochs = 50,callbacks=[early_stop,reduce_lr])

Epoch 1/50
73/73 [==============================] - 293s 4s/step - loss: 2.2538 - accuracy: 0.1676 - f1_score: 0.1610 - lr: 1.0000e-04
Epoch 2/50
73/73 [==============================] - 290s 4s/step - loss: 2.1987 - accuracy: 0.1401 - f1_score: 0.1329 - lr: 1.0000e-04
Epoch 3/50
73/73 [==============================] - 295s 4s/step - loss: 2.1698 - accuracy: 0.1731 - f1_score: 0.1664 - lr: 1.0000e-04
Epoch 4/50
73/73 [==============================] - 291s 4s/step - loss: 2.1777 - accuracy: 0.1758 - f1_score: 0.1702 - lr: 1.0000e-04
Epoch 5/50
73/73 [==============================] - 294s 4s/step - loss: 2.0682 - accuracy: 0.1923 - f1_score: 0.1839 - lr: 1.0000e-04
Epoch 6/50
73/73 [==============================] - 291s 4s/step - loss: 2.0476 - accuracy: 0.2143 - f1_score: 0.2055 - lr: 1.0000e-04
Epoch 7/50
73/73 [==============================] - 296s 4s/step - loss: 2.0850 - accuracy: 0.1566 - f1_score: 0.1446 - lr: 1.0000e-04
Epoch 8/50
73/73 [==============================] - 294

In [0]:
pred = model.predict(test)
pred = np.argmax(pred, axis=1)
test_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/dl_comptetion/dataset/test.csv")

pred = encoder.inverse_transform(pred)
result = pd.DataFrame(pred, test_data["Image"], columns=["target"])
result.to_csv("/content/drive/My Drive/Colab Notebooks/dl_comptetion/dataset/sample.csv")